In [7]:
import os
import json

In [8]:
from datasets import load_dataset 

dataset = load_dataset("multi_news",split="train")

os.mkdir("./Dataset")

Found cached dataset multi_news (C:/Users/Francisco/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


In [28]:
dataset.to_json("C:/Users/Francisco/Desktop/SmartNews/nlp_section/Dataset/train.json")

Creating json from Arrow format: 100%|██████████| 45/45 [00:04<00:00,  9.63ba/s]


571553657

<h1 style="color:red"> <b> Usar para retirar unicode </h1>

In [33]:
data = []
with open("./Dataset/train.json") as file:
    for line in dataset:
            jsonObj = json.loads(line)
            document = jsonObj["document"]
            summary = jsonObj["summary"]
            data.append((document, summary))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [34]:
from tokenizers import BertWordPieceTokenizer

wordPiece = BertWordPieceTokenizer()

wordPiece.train("./Dataset/train.json")

In [35]:
#os.mkdir('./tokenizer')

wordPiece.save_model('./tokenizer')

['./tokenizer\\vocab.txt']

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./tokenizer')

c:\Users\Francisco\Desktop\SmartNews\nlp_section\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape) #todos os valores estao entre 0 e 1
    mask_arr =(rand < 0.15) * (tensor > 2 ) #mask tokens com crit abaixo de 15% e q n usa os special tokens
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 5
    return tensor

<h1>Apagar</h1>

In [6]:
with open('./Dataset/test.json','r',encoding='utf-8') as fp:
    lines = fp.read().split('\n')


sample = tokenizer(lines,padding='max_length', truncation=True, max_length=512)



In [7]:
labels = torch.tensor([x for x in sample['input_ids']])
mask = torch.tensor([x for x in sample['attention_mask']])


In [8]:
input_ids = labels.detach().clone()
rand = torch.rand(input_ids.shape)
mask_arr = (rand < .15) * (input_ids > 2)
for i in range(input_ids.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    input_ids[i,selection] = 5 #[MASK] token

In [11]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}


In [13]:
dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [14]:
from transformers import BertConfig

config = BertConfig(vocab_size=30000, 
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1)

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(config)